In [1]:
import numpy as np
import re
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report


In [2]:
fake_news= pd.read_csv('Fake.csv')
real_news = pd.read_csv('True.csv')

In [3]:
fake_news.head()

,title,text,subject,date
0,Fake News 1,This is a fake news article number 1.,sports,07-04-2021
1,Fake News 2,This is a fake news article number 2.,politics,30-03-2020
2,Fake News 3,This is a fake news article number 3.,politics,27-04-2020
3,Fake News 4,This is a fake news article number 4.,health,25-11-2021
4,Fake News 5,This is a fake news article number 5.,economy,10-09-2019


In [4]:
real_news.head()

,title,text,subject,date
0,True News 1,This is a real news article number 1.,health,04-10-2021
1,True News 2,This is a real news article number 2.,world,07-04-2021
2,True News 3,This is a real news article number 3.,politics,14-11-2020
3,True News 4,This is a real news article number 4.,health,18-05-2019
4,True News 5,This is a real news article number 5.,health,22-01-2019


In [5]:
fake_news.shape, real_news.shape

((1000, 4), (1000, 4))

In [6]:
fake_news["class"] = 0
real_news["class"] = 1

In [7]:
fake_news_manual_testing = fake_news.head(10)
for i in range(999,989,-1):
   fake_news.drop([i],axis=0, inplace=True)
    
real_news_manual_testing = real_news.tail(10)
for i in range(999,989,-1):
   real_news.drop([i], axis=0 ,inplace=True)

In [8]:
news_manual_testing = pd.concat([fake_news_manual_testing,real_news_manual_testing],axis=0)
news_manual_testing.to_csv("manual_testing.csv")

In [9]:
news_merge=pd.concat([fake_news,real_news],axis=0)
news_merge.head(10)

,title,text,subject,date,class
0,Fake News 1,This is a fake news article number 1.,sports,07-04-2021,0
1,Fake News 2,This is a fake news article number 2.,politics,30-03-2020,0
2,Fake News 3,This is a fake news article number 3.,politics,27-04-2020,0
3,Fake News 4,This is a fake news article number 4.,health,25-11-2021,0
4,Fake News 5,This is a fake news article number 5.,economy,10-09-2019,0
5,Fake News 6,This is a fake news article number 6.,politics,16-05-2020,0
6,Fake News 7,This is a fake news article number 7.,health,09-09-2021,0
7,Fake News 8,This is a fake news article number 8.,health,22-02-2020,0
8,Fake News 9,This is a fake news article number 9.,health,12-01-2021,0
9,Fake News 10,This is a fake news article number 10.,world,13-04-2020,0


In [10]:
news= news_merge.drop(["title","subject","date"],axis=1)
news.head(10)

,text,class
0,This is a fake news article number 1.,0
1,This is a fake news article number 2.,0
2,This is a fake news article number 3.,0
3,This is a fake news article number 4.,0
4,This is a fake news article number 5.,0
5,This is a fake news article number 6.,0
6,This is a fake news article number 7.,0
7,This is a fake news article number 8.,0
8,This is a fake news article number 9.,0
9,This is a fake news article number 10.,0


In [11]:
news = news.sample(frac=1)

In [12]:
news.head(10)

,text,class
384,This is a real news article number 385.,1
49,This is a real news article number 50.,1
185,This is a fake news article number 186.,0
732,This is a real news article number 733.,1
602,This is a real news article number 603.,1
458,This is a real news article number 459.,1
0,This is a fake news article number 1.,0
923,This is a real news article number 924.,1
123,This is a real news article number 124.,1
508,This is a real news article number 509.,1


In [13]:
news.isnull().sum()

text     0
class    0
dtype: int64

In [14]:
import re
import string

def word_drop(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r'\w*\d\w*', '', text)
    return text

In [15]:
import pandas as pd
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")


In [16]:
# Label the datasets
fake["label"] = 0  # Fake news
true["label"] = 1  # Real news

# Combine both into one DataFrame
news = pd.concat([fake, true], ignore_index=True)


In [17]:
news['text'] = news['text'].apply(word_drop)


In [18]:
news.head(10)

,title,text,subject,date,label
0,Fake News 1,this is a fake news article number,sports,07-04-2021,0
1,Fake News 2,this is a fake news article number,politics,30-03-2020,0
2,Fake News 3,this is a fake news article number,politics,27-04-2020,0
3,Fake News 4,this is a fake news article number,health,25-11-2021,0
4,Fake News 5,this is a fake news article number,economy,10-09-2019,0
5,Fake News 6,this is a fake news article number,politics,16-05-2020,0
6,Fake News 7,this is a fake news article number,health,09-09-2021,0
7,Fake News 8,this is a fake news article number,health,22-02-2020,0
8,Fake News 9,this is a fake news article number,health,12-01-2021,0
9,Fake News 10,this is a fake news article number,world,13-04-2020,0


In [19]:
x = news["text"]
y = news["label"]

In [20]:
x_train , x_test , y_train , y_test = train_test_split(x,y, test_size = .25)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
vectorisation = TfidfVectorizer()
xv_train = vectorisation.fit_transform(x_train)
xv_test = vectorisation.transform(x_test)

logistic Regression

In [23]:
from sklearn.linear_model import LogisticRegression

In [24]:
LR = LogisticRegression()
LR.fit(xv_train, y_train)

LogisticRegression()

In [25]:
LR.score(xv_test, y_test)


1.0

In [26]:
pred_LR = LR.predict(xv_test)
print(classification_report(y_test, pred_LR))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00       266

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



Decision Tree classification

In [40]:
from sklearn.tree import DecisionTreeClassifier
DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)
print("Accuracy:", DT.score(xv_test, y_test))

Accuracy: 1.0


In [41]:
DT.score(xv_test, y_test)


1.0

In [42]:
pred_DT = DT.predict(xv_test)
print(classification_report(y_test, pred_DT))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00       266

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



Gradient Boosting Classifier

In [30]:
from sklearn.ensemble import GradientBoostingClassifier

In [31]:
GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

In [32]:
GBC.score(xv_test, y_test)

1.0

In [33]:
pred_GBC = GBC.predict(xv_test)
print(classification_report(y_test, pred_GBC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00       266

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



Random Forest Classifier

In [34]:
from sklearn.ensemble import RandomForestClassifier

In [35]:
RFC = RandomForestClassifier()
RFC.fit(xv_train, y_train)

RandomForestClassifier()

In [36]:
RFC.score(xv_test, y_test)

1.0

In [37]:
pred_RFC = RFC.predict(xv_test)
print(classification_report(y_test, pred_RFC))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       234
           1       1.00      1.00      1.00       266

    accuracy                           1.00       500
   macro avg       1.00      1.00      1.00       500
weighted avg       1.00      1.00      1.00       500



Manual Testing 

In [38]:
def output_label(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text": [news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(word_drop)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorisation.transform(new_x_test)

    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\nLR Prediction: {}\nDT Prediction: {}\nGBC Prediction: {}\nRFC Prediction: {}".format(
        output_label(pred_LR[0]),
        output_label(pred_DT[0]),
        output_label(pred_GBC[0]),
        output_label(pred_RFC[0])
    ))


In [43]:
news = str(input()) 
manual_testing(news)


LR Prediction: Fake News
DT Prediction: Not A Fake News
GBC Prediction: Not A Fake News
RFC Prediction: Not A Fake News
